# Maximum Likelihood Estimation 

MLE is used to maximize the likelihood of the joint probability of the apperance of the data.

In [17]:
import numpy as np
import scipy.optimize as opt
from scipy.stats import norm, t

def mle_fit(y, X, model_type):
    def ll_normal(params, X, y):
        b0, b1, e = params
        y_pred = b0 + X * b1
        residuals = y - y_pred
        likelihood = np.sum(norm.logpdf(residuals, loc = 0, scale = e)) #e是error的标准差
        return -likelihood

    def ll_regression(params, X, y):
        s = params[0]
        beta = params[1:]
        e = y - np.dot(X, beta)
        return -np.sum(norm.logpdf(e, loc=0, scale=s))

    def ll_t(params, X, y):
        b0, b1, e, df = params
        y_pred = b0 + b1 * X
        residuals = y - y_pred
        likelihood = np.sum(t.logpdf(residuals, df, loc = 0, scale = df))
        return -likelihood

    if model_type == 'normal':
        result = opt.minimize(ll_normal, [0.0, 1.0, 1.0], args=(X[:,1], y), method='Nelder-Mead')
    elif model_type == 'regression':
        initial_guess = [1.0] + [0.0] * (X.shape[1])
        result = opt.minimize(ll_regression, initial_guess, args=(X[:,1], y), method='Nelder-Mead')
    elif model_type == 't':
        result = opt.minimize(ll_t, [10, 0.0, 1.0, 1.0], args=(X[:,1], y), method='Nelder-Mead')
    else:
        raise ValueError("Invalid model type")

    return result
# x：找到的最优解，即最小化目标函数时的参数值。
# fun：在最优解处的目标函数值。
# success：一个布尔值，表示优化过程是否成功。
# message：关于优化过程的附加信息或退出状态的说明。
# nit：执行的迭代次数。
# jac（如果适用）：在最优解处的目标函数的梯度。
# hess_inv（如果适用）：最优解处的目标函数的海森矩阵的逆。

In [18]:
import pandas as pd

data = pd.read_csv('problem2_545.csv')
X = data['x'].values
y = data['y'].values
# Adding a constant term
X = np.column_stack((np.ones(len(X)), X))

mle_fit(y, X, 't')

 final_simplex: (array([[ -0.09034869,   0.57201994, -28.06004637,   0.87446691],
       [ -0.09034751,   0.57202031, -28.06006575,   0.87446626],
       [ -0.09034859,   0.57201816, -28.05995573,   0.87446666],
       [ -0.0903476 ,   0.57201975, -28.06003646,   0.87446746],
       [ -0.09034818,   0.57201819, -28.05995664,   0.87446748]]), array([327.95928387, 327.95928387, 327.95928387, 327.95928387,
       327.95928387]))
           fun: 327.95928386987214
       message: 'Optimization terminated successfully.'
          nfev: 457
           nit: 266
        status: 0
       success: True
             x: array([ -0.09034869,   0.57201994, -28.06004637,   0.87446691])

In [19]:
def calculate_information_criteria(result, n):
    """
    Calculate AIC, AICc, and BIC for a fitted model.

    :param result: The result object returned from the minimize function.
    :param n: The number of observations in the dataset.
    :return: A tuple containing the AIC, AICc, and BIC values.
    """
    k = len(result.x)  # The number of estimated parameters
    ll = -result.fun  # The log-likelihood of the model
    aic = 2 * k - 2 * ll
    aicc = aic + (2 * k**2 + 2 * k) / (n - k - 1)
    bic = np.log(n) * k - 2 * ll
    
    return aic, aicc, bic    

In [20]:
result = mle_fit(y, X, 't')
n = len(X)
calculate_information_criteria(result, n)

(663.9185677397443, 664.1236959448725, 677.1118372059365)